In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
URL = 'https://www.tradeindia.com/manufacturers/bio-fertilizers.html'

In [3]:
HEADERS = {
    'User-Agent': 'add_your_user_agent', 
    'Accept-Language': 'en-US, en;q=0.5'
}

In [4]:
def fetch_page(url):
    response = requests.get(url, headers=HEADERS)
    return response

In [5]:
def parse_product_links(page_content):
    soup = BeautifulSoup(page_content, 'html.parser')
    links = soup.find_all('a', attrs={'class':'product-title-position'})
    return [link.get('href') for link in links]

In [9]:
def fetch_product_details(url):
    page = fetch_page(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    details = {}

    # Safe extraction function
    def safe_find(tag, attrs):
        element = soup.find(tag, attrs=attrs)
        return element.text.strip() if element else 'Not Available'

    details['product_name'] = safe_find('h1', {'class':'sc-99c38e0-3 crggkU m-0 Headline3'})
    details['price'] = safe_find('h2', {'class':'sc-99c38e0-3 kpwqmK prPrice Headline3'})
    details['rating'] = safe_find('span', {'class':'mr-1'})
    details['company'] = safe_find('h2', {'class':'sc-99c38e0-1 dtjLEg co-name'})
    details['contact_name'] = safe_find('p', {'class':'sc-99c38e0-13 bFlIdM m-0 mb-3 Body4R'})

    # Address might require special handling if it's not always in the third element
    elements = soup.find_all('p', attrs={'class': 'sc-99c38e0-13 bFlIdM m-0 mb-3 Body4R'})
    details['address'] = elements[2].text.strip() if len(elements) >= 3 else 'Not Available'

    return details

In [10]:
main_page = fetch_page(URL)
product_urls = parse_product_links(main_page.content)

In [11]:
all_products = []
for url in product_urls:
    product_details = fetch_product_details(url)
    all_products.append(product_details)

In [12]:
df = pd.DataFrame(all_products)

In [13]:
df.to_csv('bio_fertilizers.csv', index=False)